In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


If you want to get something done you should start from the start. This is Day 1 of orientation microscopy. We create a fake pattern. We create a library that contains the fake pattern, and some other patterns. We check the process works as expected. This is for multiple DP's now

In [69]:
%matplotlib tk
import numpy as np
import pyxem as pxm
from matplotlib import pyplot as plt
import hyperspy.api as hs

In [70]:
from pyxem.diffraction_generator import DiffractionSimulation
from pyxem.indexation_generator import IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

Fake patterns get made

In [71]:
dps = []
dp_sim_list = []
for alpha in [0,1,2,3]:
    coords = (np.random.rand(5,2)-0.5)*2 #zero mean, range from -1 to +1
    dp_sim = DiffractionSimulation(coordinates=coords,intensities=np.ones_like(coords[:,0]),calibration=1/72)
    dp_sim_list.append(dp_sim)
    dpx = dp_sim.as_signal(144,0.075,1).data
    dps.append(dpx)

dp = pxm.ElectronDiffraction([dps[0:2],dps[2:]])


And a fake library

In [72]:
library = dict()
library["Phase"] = {}
for alpha in np.arange(0,5,1):
    rotation = (alpha,0,0)
    if rotation[0] < 4:
        library["Phase"][rotation] = dp_sim_list[rotation[0]]
    else:
        local_cords = np.random.rand(5,2)
        library["Phase"][rotation] = DiffractionSimulation(coordinates=local_cords,intensities=np.ones_like(local_cords[:,0]))

In [73]:
indexer = IndexationGenerator(dp,library)

In [74]:
match_results = indexer.correlate()

A Jupyter Widget

In [76]:
peaks = match_results.map(peaks_from_best_template,phase=["Phase"],library=library,inplace=False)

A Jupyter Widget

In [78]:
mmx,mmy = generate_marker_inputs_from_peaks(peaks)
dp.set_calibration(2/144)
dp.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=my,y=mx,color='red',marker='x')
    dp.add_marker(m,plot_marker=True,permanent=True)